In [6]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.6,
    model="glm-4-0520",
    openai_api_key="661a7aa0aeb8ca129eb4647461123230.bl9w581QKpnMfBvs",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)


In [6]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call


In [19]:
from typing import List

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    height_in_meters: float = Field(
        ..., description="The height of the person expressed in meters."
    )


# Set up a parser
parser = PydanticOutputParser(pydantic_object=Person)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [ ]:
PydanticOutputParser(pydantic_object=Person).get_format_instructions()

In [ ]:
query = "Anna is 23 years old and she is 6 feet tall"

print(prompt.invoke(query).to_string())

In [13]:
chain = prompt | llm | parser

z = chain.invoke({"query": query})

In [1]:
from typing import List
from typing import Literal, Optional, Tuple
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field


class Soybean_varinfor(BaseModel):
    """Information about a person."""

    name: str = Field(
        ..., description="大豆的品种名称"
    )
    yield_soy: float = Field(
        ..., description="大豆的产量信息(kg/ha)"
    )
    shengyuqi: int = Field(
        ...,
        description="根据用户提出的问题，回答品种平均生育期(天)",
    )
    pingzhong: Literal["大豆", "牧草", "玉米"] = Field(
        ...,
        description="根据用户提出的问题，选择最相关的农业作物类型来源以回答他们的问题",
    )
    yield_ofsoy_100: Optional[float] = Field(
        None,
        description="百粒重（克）",
    )
    

# Set up a parser
parser = PydanticOutputParser(pydantic_object=Soybean_varinfor)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())


In [2]:
query = "北豆40，为产于黑龙江省的大豆种子。是由黑龙江省农垦总局红兴隆科学研究所与黑龙江省农垦科研育种中心共同研发的产品。品种特性该品种平均生育期120天，长叶、紫花、亚有限结荚习性。株高85.8厘米，单株有效荚数34.4个，百粒重19.0克。籽粒圆形，种皮黄色，黄脐。接种鉴定，中抗大豆灰斑病，中抗SMVⅠ号株系，中感SMVⅢ号株系。粗蛋白质含量40.78%，粗脂肪含量21.99%。产量表现2006年参加北方春大豆中早熟组品种区域试验，亩产210.6千克，比对照绥农14增产4.0%，极显著；2007年续试，亩产181.2千克，比对照增产6.5%，极显著；两年区域试验亩产195.9千克，比对照增产5.2%。2007年生产试验，亩产166.8千克，比对照增产5.2%。栽培技术要点地温稳定通过7～8℃开始播种，适宜种植密度为每亩1.6万～1.7万株；以深秋施肥为好，每亩施纯量化肥8～10千克，氮、磷、钾比例1∶1.15～1.5∶0.8。该品种符合国家大豆品种审定标准，通过审定。适宜在黑龙江省第二积温带和第三积温带上限，吉林省东部地区春播种植。"

print(prompt.invoke(query).to_string())

System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Information about a person.", "properties": {"name": {"title": "Name", "description": "\u5927\u8c46\u7684\u54c1\u79cd\u540d\u79f0", "type": "string"}, "yield_soy": {"title": "Yield Soy", "description": "\u5927\u8c46\u7684\u4ea7\u91cf\u4fe1\u606f(kg/ha)", "type": "number"}, "shengyuqi": {"title": "Shengyuqi", "description": "\u6839\u636e\u7528\u6237\u63d0\u51fa\u7684\u95ee\u9898\uff0c\u56de\u7b54\u54c1\u79cd\u5e73\u5747\u751f\u80b2\u671f(\u5929)", "type": "integer"},

In [5]:
chain = prompt | llm | parser

z = chain.invoke({"query": query})
z

Soybean_varinfor(name='北豆40', yield_soy=195.9, shengyuqi=120, pingzhong='大豆', yield_ofsoy_100=19.0)